In [1]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
# Imports - may not all 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from tensorflow.keras.backend import expand_dims
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

In [3]:
!pwd

/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/notebooks


In [7]:
# Importing data, normalising data, (not augmenting data - hashed out), for train and val set 
datagen = ImageDataGenerator(
    rescale = 1./255, 
    # rotation_range = 5,
    # zoom_range = 0.1,
    # brightness_range = (0.95, 0.95),
    # horizontal_flip = True,
    # vertical_flip = True,
    data_format = 'channels_last',
    validation_split = 0.2,
    dtype = tf.float32
)

train_generator = datagen.flow_from_directory(
    '/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train',
    target_size = (224, 224),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training',
    seed = 123
)

validation_generator = datagen.flow_from_directory(
    '/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train',
    target_size = (224, 224),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'validation',
    seed = 123
)

Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.


In [ ]:
# Importing data, normalising data, (not augmenting data - hashed out), for test set 
datagen2 = ImageDataGenerator(
    rescale = 1./255, 
    # rotation_range = 5,
    # zoom_range = (0.90, 0.90),
    # brightness_range = (0.95, 0.95),
    # horizontal_flip = True,
    # vertical_flip = True,
    data_format = 'channels_last',
    validation_split = 0.0,
    dtype = tf.float32
)

test_generator = datagen2.flow_from_directory(
    '/content/drive/MyDrive/LeWagon/raw_data/AlzheimersDataset/test',
    target_size = (224, 224),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    seed = 123
)

In [ ]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# train_generator = train_generator.cache().prefetch(buffer_size=AUTOTUNE)
# validation_generator = validation_generator.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# # THE MODELLLLLL
# DenseNet121 model
base_model = DenseNet121(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

# Compiling
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['AUC', 'accuracy', 'Recall', 'Precision'])

# Early stopping criterion
# es = EarlyStopping(patience=15, restore_best_weights=True)
rop = ReduceLROnPlateau(monitor='val_loss', factor=0.005, patience=15, restore_best_weights=True, min_lr=0.005)

In [ ]:
# Fitting the model
model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[rop], verbose=1)

In [25]:
# Test against validation generator
model.evaluate(validation_generator)

32/32 [==============================] - 6s 180ms/step - loss: 1.0879 - auc: 0.8369 - accuracy: 0.5503 - recall: 0.5239 - precision: 0.5589


[1.0879493951797485,
 0.8369439840316772,
 0.5503421425819397,
 0.523949146270752,
 0.5589155554771423]

In [26]:
# Test against test generator
model.evaluate(test_generator)

40/40 [==============================] - 7s 178ms/step - loss: 1.3454 - auc: 0.8058 - accuracy: 0.5160 - recall: 0.5082 - precision: 0.5263


[1.3453662395477295,
 0.8057735562324524,
 0.5160281658172607,
 0.5082095265388489,
 0.5263158082962036]

In [ ]:
# Confusion matrix with validation generator

y_pred = model.predict(validation_generator)

CLASSES = [ 'MildDemented',
            'ModerateDemented',
            'NonDemented',
            'VeryMildDemented']

conf_arr = confusion_matrix(validation_generator.classes, np.argmax(y_pred, axis=-1))

plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=CLASSES, yticklabels=CLASSES)

plt.title('Alzheimer\'s Diagnosis')
plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.show(ax)

In [ ]:
# Confusion matrix with test generator

y_pred = model.predict(test_generator)

CLASSES = [ 'MildDemented',
            'ModerateDemented',
            'NonDemented',
            'VeryMildDemented']

conf_arr = confusion_matrix(test_generator.classes, np.argmax(y_pred, axis=-1))

plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=CLASSES, yticklabels=CLASSES)

plt.title('Alzheimer\'s Diagnosis')
plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.show(ax)